In [20]:
# for debugging the jcapper loader code, run this script "python -i load_jcapper.py"
import sys
sys.path.append('I:/YaoTony/code/x8313')
sys.path.append('/Users/saleemhuda/Dropbox/X8313_Live')
from datetime import date
import pytest
from pandas import date_range
import pandas as pd
from horse.betsim import data
import os.path
track_detail = os.path.join(data.__path__._path[0], 'track_detail.csv')
df_track_detail = pd.read_csv(track_detail, index_col=['x8_track_sym'])
map_jcp_itsp = df_track_detail.set_index('jcp_track_sym')['itsp_track_sym'].to_dict()
map_rebates_win = df_track_detail.WN.to_dict()
df_hist = pd.read_csv('./dfX_hist.csv')
df_hist['payout_win'] = df_hist['final_tote_odds']+1
df_hist['rebate_win'] = df_hist['track_id'].map(lambda x:map_rebates_win.get(x,0.0))


In [5]:
#Functions for adding arbitrary sets of filtering criteria
def merge_dicts(*dict_args):
    """
    Given any number of dicts, shallow copy and merge into a new dict,
    precedence goes to key value pairs in latter dicts.
    """
    result = {}
    for dictionary in dict_args:
        result.update(dictionary)
    return result

def apply_criteria(df, dict_crit):
    #apply dicts in dict_crit
    #Applies criteria to filter runners in df'
    #i.e.'x8is_par_HDWPSRRating': lambda df:df[df['x8is_HDWPSRRating_norm_par']>0]
    for crit_name, crit_func in dict_crit.items():
        print("applying crit {}".format(crit_name))
        print("num_runners before: {}".format(len(df.runner_id.unique()))) #TODO: add race count, track count 
        df = crit_func(df)
        print("num_runners after: {}".format(len(df.runner_id.unique())))
    return df

def get_rebates_by_track_id(df, default_rebate=0.0):
    """
    can use track_detail.csv
    
    """
    
    pass

    
def compute_simple_payout(df, attr_model, ascending=False, bet_amount=1.0):
    """
    Add columns for quick calculation of Win bets % payout
    :param df: Dataframe from dataset for multiple races
    :param attr_model: (string)an attribute / prob or score in the dataframe that can be ranked
    :param ascending: False if best is higher score i.e. probs
    :param bet_amount: default to 1.0
    :return: Dataframe with columns added

    """
    df['is_win'] = df['official_finish_position'].map(lambda x:int(x==1)) #1 if this runner is winner 
    #add rank attribute for model 
    df['rank_' + attr_model] = df.groupby('race_id')[attr_model].transform(lambda x:x.rank(ascending=ascending))
    df['bet_amount'] = df['rank_' + attr_model].map(lambda x:int(x<1.5) * bet_amount) #we bet 1 unit if rank is 1 
    df['is_wager'] = df['bet_amount'].map(lambda x:int(x>0)) #indicator for rows with bets
    df['is_paid'] = df['is_wager'] * df['is_win']
    #This is the payout (without impact) of the 
    df['itsp_track_sym'] = df['track_id'].map(lambda x: map_jcp_itsp.get(x,x))
    
    
    df['payout'] = df['is_win']*df['bet_amount']*df['payout_win'].fillna(0.0) + df['bet_amount']*df['rebate_win']#rebate
    
    return df

def compute_advantage(dfpay,rebate=0.0):
    #Computes the pct advantage as:
    #Win% - (Loss% / (mean_odds + rebate))
    #dfpay: comes from compute_simple_payout() 
    #rebate 
    pct_win = dfpay.groupby('race_id')['is_paid'].sum().value_counts(normalize=True)[1]

    pct_loss = 1.0 - pct_win
    mean_odds = (dfpay[dfpay.is_paid>0]['final_tote_odds'] + dfpay[dfpay.is_paid>0]['rebate_win']).mean()
    advantage = pct_win  - pct_loss / (mean_odds+rebate)
    print(advantage)
    return advantage


In [6]:
attr_bench = 'prob_x8runner_HDWPSRRating_norm_par'
rbt = 0.0
#since advantage is nothing about bet_amount, we use the original functions without bet_amount type.
df_payout_bench = compute_simple_payout(df_hist, attr_model = attr_bench, ascending=False, bet_amount=1.0)
advantage_bench = compute_advantage(df_payout_bench)

-0.13199672654902644


## PRE: Define criteria for filtering after 12:00 pm
### This can be done by loading races from daily.DailyRaces() class
### NEED TO map attributes

In [7]:
tracks_all = df_hist.track_id.unique()

In [8]:
tracks_all

array(['APX', 'BEL', 'BTP', 'CBY', 'DEL', 'ELP', 'EMD', 'FLX', 'LAD',
       'LSX', 'NPX', 'PHA', 'PID', 'PLN', 'PRM', 'RUI', 'SAX', 'TDN',
       'FEX', 'FPK', 'GPX', 'GRP', 'IND', 'ALB', 'ASD', 'EVD', 'PEN',
       'CTX', 'FMT', 'LRC', 'ARP', 'HPO', 'LAX', 'LRL', 'MDX', 'MTH',
       'GPR', 'HST', 'SUF', 'WYO'], dtype=object)

In [9]:
today = pd.datetime.today().date()

In [10]:
today_weekday = today.weekday()

In [11]:
#Here we create dictionaries for the criteria we can apply using information available before betting starts

#Filter tracks to bettable tracks
dict_criteria_weekday_pre = {'weekday':lambda df:df[df['weekday']==today_weekday] }
#dict_criteria_track = {'test_tracks': lambda df:df[df['track_id'].isin(['SUN','OPX','HAW','APX','EMD','FEX','PID','PHA','PEN','ASD','EVD','BTP','DEL','LAD','BEL','WOX','MNR','CTX','CBY'])]}
                       #'B_tracks': lambda df:df[df['track_id'].isin(['LAD','PEN','PHA'])
dict_criteria_race_pre = {'x8_num_starters_pp_9': lambda df:df[df['x8_num_starters_pp']>8]}

dict_criteria_runner_pre = {'x8is_par_HDWPSRRating': lambda df:df[df['x8is_HDWPSRRating_norm_par']>0],
                            'x8is_top4_ml': lambda df:df[df['rank_morning_line']<5]}

dict_criteria_odds_play = {'x8is_top4_final': lambda df:df[df['rank_final_tote_odds']<5]}
dict_criteria_odds_final = {'x8is_favorite_odds': lambda df:df[df['rank_final_tote_odds']<1.5]}


In [17]:
#criteria using available information prior to racetime
#use DailyRaces(status='pre')
dict_crit_pre = merge_dicts(dict_criteria_weekday_pre, #limit to weekday today 0 Monday, 1 Tuesday
                            dict_criteria_race_pre, #race level criteria such as num runners at least 'x8_num_starters_pp > 7'
                            dict_criteria_runner_pre) #runner level criteria available prior to race morningline, HDWPSRRating

dict_crit_play = merge_dicts(dict_crit_pre, 
                             dict_criteria_odds_play)

In [18]:
df_filter_pre = apply_criteria(df_hist, dict_crit_pre )
df_filter_play = apply_criteria(df_hist, dict_crit_play )

applying crit x8is_top4_ml
num_runners before: 7406
num_runners after: 3826
applying crit x8_num_starters_pp_9
num_runners before: 3826
num_runners after: 1221
applying crit x8is_par_HDWPSRRating
num_runners before: 1221
num_runners after: 521
applying crit weekday
num_runners before: 521
num_runners after: 59
applying crit weekday
num_runners before: 7406
num_runners after: 587
applying crit x8is_top4_final
num_runners before: 587
num_runners after: 283
applying crit x8_num_starters_pp_9
num_runners before: 283
num_runners after: 116
applying crit x8is_par_HDWPSRRating
num_runners before: 116
num_runners after: 60
applying crit x8is_top4_ml
num_runners before: 60
num_runners after: 51


In [19]:
attr = attr_bench
df_payout_pre = compute_simple_payout(df_filter_pre, attr_model = attr, ascending=False, bet_amount=1.0)
df_payout_play = compute_simple_payout(df_filter_play, attr_model = attr, ascending=False, bet_amount=1.0)
advantage_pre = compute_advantage(df_payout_pre)
advantage_play = compute_advantage(df_payout_play)

0.08000217280675176
0.10698877573775367


In [ ]:
df_payout_pre.info()

In [ ]:
df_payout_play.info()

In [ ]:
df_payout_pre.head()

In [ ]:
df_payout_play.tail()